In [1]:
import config as cfg
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from pathlib import Path
from sklearn.model_selection import train_test_split
import pickle

### Load the train, validation, test sets

In [2]:
data = pd.read_pickle(cfg.PATHS.TRANSFORMED_DATA_PICKLE)
data.shape

(139805, 931)

In [3]:
# only keep rows with more than 10 reactions "funny", "useful", "cool"
data = data[data["funny"] + data["useful"] + data["cool"] > 10]
data.shape

(4563, 931)

In [4]:

# Split the data into training and testing sets
X = data.drop(columns=["useful", "funny", "cool"], axis=1)
y = data[["useful", "funny", "cool"]]

del data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

del X
del y

### Build the model

In [5]:
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                59456     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 3)                 51        
                                                                 
Total params: 62,115
Trainable params: 62,115
Non-trainable params: 0
_________________________________________________________________


2023-03-25 15:36:05.627673: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-25 15:36:05.627999: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
history = model.fit(
    np.array(X_train),
    np.array(y_train),
    epochs=10,
    batch_size=32,
    validation_split=0.2,
)

Epoch 1/10


2023-03-25 15:36:09.971963: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-25 15:36:10.263563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


92/92 [==============================] - ETA: 0s - loss: 45491.6016 - acc: 0.8329

2023-03-25 15:36:12.017322: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


92/92 [==============================] - 2s 18ms/step - loss: 45491.6016 - acc: 0.8329 - val_loss: 128960.6172 - val_acc: 0.8178
Epoch 2/10
92/92 [==============================] - 1s 15ms/step - loss: 380519.3750 - acc: 0.8356 - val_loss: 648376.8750 - val_acc: 0.8178
Epoch 3/10
92/92 [==============================] - 1s 15ms/step - loss: 1271094.6250 - acc: 0.8356 - val_loss: 1833450.6250 - val_acc: 0.8178
Epoch 4/10
92/92 [==============================] - 1s 15ms/step - loss: 3075201.5000 - acc: 0.8356 - val_loss: 3925080.2500 - val_acc: 0.8178
Epoch 5/10
92/92 [==============================] - 1s 15ms/step - loss: 5947738.0000 - acc: 0.8356 - val_loss: 6793737.5000 - val_acc: 0.8178
Epoch 6/10
92/92 [==============================] - 1s 15ms/step - loss: 9325046.0000 - acc: 0.8356 - val_loss: 10067414.0000 - val_acc: 0.8178
Epoch 7/10
92/92 [==============================] - 1s 14ms/step - loss: 13163964.0000 - acc: 0.8356 - val_loss: 13826747.0000 - val_acc: 0.8178
Epoch 8/10
9

In [7]:
model.evaluate(np.array(X_test), np.array(y_test))

29/29 [==============================] - 0s 9ms/step - loss: 37014768.0000 - acc: 0.0778


[37014768.0, 0.07776561379432678]

In [11]:
# find indices of all rows in test data with greater than 10 votes in any category
model.predict(np.array(X_test))

29/29 [==============================] - 0s 3ms/step


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [9]:
pickle.dump(model, cfg.PATHS.KERAS_MODEL.open("wb"))

INFO:tensorflow:Assets written to: ram://49075aba-efdc-4188-ba56-8b6c34550918/assets
